Import feature files and load them to a FeatureMap() file

In [1]:
import os
import glob
import sys
#sys.path.insert(0, '/Users/eeko/openms-develop/openms_build/pyOpenMS')
from pyopenms import *
import pandas as pd

Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


In [2]:
path= "results/GNPSexport/interim/"
isExist= os.path.exists(path)
if not isExist:
    os.mkdir(path)

input_feature_files = glob.glob('results/features/interim/*.featureXML')
feature_maps = []
for featurexml_file in input_feature_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(featurexml_file, fmap)
    feature_maps.append(fmap)

1. MapAlignmentAlgorithmPoseClustering algorithm is used to align the retention time shifts caused by chromatographic differences. 
The reference file used for Map Alignment is the feature map with the highest number of features.

In [15]:
ref_index = [i[0] for i in sorted(enumerate([fm.size() for fm in feature_maps]), key=lambda x:x[1])][-1]

aligner = MapAlignmentAlgorithmPoseClustering()
aligner_par= aligner.getDefaults()

aligner_par.setValue("max_num_peaks_considered", -1)
aligner_par.setValue("superimposer:mz_pair_max_distance", 0.05)
aligner_par.setValue("pairfinder:distance_MZ:max_difference", 10.0)
aligner_par.setValue("pairfinder:distance_MZ:unit", "ppm")
aligner.setParameters(aligner_par)
aligner.setReference(feature_maps[ref_index])

for feature_map in feature_maps[:ref_index] + feature_maps[ref_index+1:]:
    trafo = TransformationDescription()
    aligner.align(feature_map, trafo)
    transformer = MapAlignmentTransformer()
    transformer.transformRetentionTimes(feature_map, trafo, True) # store original RT as meta value

for feature_map in feature_maps:    
    feature_file = os.path.join("results", "", "GNPSexport", "", "interim", "", 'MapAligned_' + os.path.basename(feature_map.getMetaValue('spectra_data')[0].decode())[7:-5] +".featureXML")
    trafo_file= os.path.join("results", "", "GNPSexport", "", "interim", "", 'MapAligned_' + os.path.basename(feature_map.getMetaValue('spectra_data')[0].decode())[7:-5] +".trafoXML")
    FeatureXMLFile().store(feature_file, feature_map)
    TransformationXMLFile().store(trafo_file, trafo)

Import aligned feature files and load them to a FeatureMap() file

In [4]:
input_feature_files = sorted(glob.glob("results/GNPSexport/interim/MapAligned*.featureXML"))

feature_maps = []
for featurexml_file in input_feature_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(featurexml_file, fmap)
    feature_maps.append(fmap)

2. IDMapper annotates MS2 fragmentations as peptide/protein identifications. This is the only way to currently annotate MS2 data for GNPS FBMN

In [8]:
use_centroid_rt= False
use_centroid_mz= True
protein_ids = []
peptide_ids= []

mapper = IDMapper()

input_mzml_files= sorted(glob.glob("results/interim/PCpeak_*.mzML"))

for filename in input_mzml_files:
    exp = MSExperiment()
    MzMLFile().load(filename, exp)

    for fmap in feature_maps:
        peptide_ids = []
        protein_ids = []
        if os.path.basename(fmap.getMetaValue('spectra_data')[0].decode()) == os.path.basename(filename):
            mapper.annotate(fmap, peptide_ids, protein_ids, use_centroid_rt, use_centroid_mz, exp)
            featureidx_file = os.path.join("results", "", "GNPSexport", "", "interim", "", 'IDMapper_' + os.path.basename(fmap.getMetaValue('spectra_data')[0].decode())[7:-5] +".featureXML")
            FeatureXMLFile().store(featureidx_file, fmap)

Unassigned peptides: 0
Peptides assigned to exactly one feature: 0
Peptides assigned to multiple features: 0
Unassigned and unidentified precursors: 4835
Unidentified precursor assigned to exactly one feature: 0
Unidentified precursor assigned to multiple features: 0
Feature annotation with identifications:
    no ID: 2776
    single ID: 0
    multiple IDs (identical): 0
    multiple IDs (divergent): 0


Unassigned peptides: 0
Peptides assigned to exactly one feature: 0
Peptides assigned to multiple features: 0
Unassigned and unidentified precursors: 4986
Unidentified precursor assigned to exactly one feature: 0
Unidentified precursor assigned to multiple features: 0
Feature annotation with identifications:
    no ID: 1253
    single ID: 0
    multiple IDs (identical): 0
    multiple IDs (divergent): 0


Unassigned peptides: 0
Peptides assigned to exactly one feature: 0
Peptides assigned to multiple features: 0
Unassigned and unidentified precursors: 5042
Unidentified precursor assigne

Import the annotated feature files and load them in a FeatureMap() file

In [14]:
input_feature_files = sorted(glob.glob('results/GNPSexport/interim/IDMapper*.featureXML'))

feature_maps = []
for featurexml_file in input_feature_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(featurexml_file, fmap)
    feature_maps.append(fmap)

3. The Feature Grouping Algorithm is used to aggregate the feature information (from single files) into a ConsensusFeature, linking features from different files together, which have a smiliar m/z and rt (MS1 level).

In [16]:
feature_grouper = FeatureGroupingAlgorithmKD()

consensus_map = ConsensusMap()
file_descriptions = consensus_map.getColumnHeaders()

for i, feature_map in enumerate(feature_maps):
    file_description = file_descriptions.get(i, ColumnHeader())
    file_description.filename = feature_map.getMetaValue('spectra_data')[0].decode()
    file_description.size = feature_map.size()
    file_descriptions[i] = file_description

feature_grouper.group(feature_maps, consensus_map)
consensus_map.setColumnHeaders(file_descriptions)

Consensus_file= os.path.join("results", "", "GNPSexport", "","interim", "", 'consensus' + ".consensusXML")
ConsensusXMLFile().store(Consensus_file, consensus_map)


# get intensities as a DataFrame
intensities = consensus_map.get_intensity_df()

# get meta data as DataFrame
meta_data = consensus_map.get_metadata_df()[['RT', 'mz', 'charge']]

# you can concatenate these two for a "result" DataFrame
result = pd.concat([meta_data, intensities], axis=1)

# if you don't need labeled index, remove it (and/or save with index = False)
result.reset_index(drop=True, inplace=True)

# store as tsv file
result.to_csv('results/GNPSexport/interim/consensus.tsv', sep = '\t', index = False)

Progress of 'computing RT transformations':
-- done [took 0.61 s (CPU), 0.63 s (Wall)] -- 
Progress of 'linking features':
-- done [took 1.36 s (CPU), 1.35 s (Wall)] -- 
<ConsensusXMLFile::store():  found 17978 invalid unique ids> occurred 2 times
ConsensusXMLFile::store():  found 17946 invalid unique ids


4. File-filtering is used to remove all features that do not have an MS2 spectrum.

In [2]:
input_consensus= "results/GNPSexport/interim/consensus.consensusXML"
cmap = ConsensusMap()
ConsensusXMLFile().load(input_consensus, cmap)
new_map= ConsensusMap(cmap)
new_map.clear(False)
for f in cmap:
    if f.getPeptideIdentifications() !=[]:
        new_map.push_back(f)
        
Consensus_file= os.path.join("results", "", "GNPSexport", "", "interim", "",'filtered' + ".consensusXML")
ConsensusXMLFile().store(Consensus_file, new_map)

ConsensusXMLFile::store():  found 7051 invalid unique ids


5. Convert the ConsensusXML file to a FeatureQuantificationTable.txt file compatible to GNPS

In [3]:
input_consensus= "results/GNPSexport/interim/filtered.consensusXML"
cmap = ConsensusMap()
ConsensusXMLFile().load(input_consensus, cmap)

# get intensities as a DataFrame
intensities = cmap.get_intensity_df()

# get meta data as DataFrame
meta_data = cmap.get_metadata_df()

# you can concatenate these two for a "result" DataFrame
result = pd.concat([meta_data, intensities], axis=1)

# if you don't need labeled index, remove it (and/or save with index = False)
result.reset_index(drop=True, inplace=True)

idx = 0
new_col = "CONSENSUS"  # can be a list, a Series, an array or a scalar   
result.insert(loc=idx, column='#CONSENSUS', value=new_col)   
result= result.rename(columns= {"charge": "charge_cf", "RT": "rt_cf", "mz": "mz_cf", "quality": "quality_cf", "width": "width_cf"})
result= result.drop(["sequence"], axis= 1)
result= result.sort_index(axis=1)

filemeta= cmap.getColumnHeaders()
mapIDs = [k for k in filemeta.keys()]
filename= []
size=[]
label= []
for header in filemeta.values():
    files= header.filename
    sizes= header.size
    labels= header.label
    filename.append(files)
    size.append(sizes)
    label.append(labels)

dict = {'id': mapIDs, 'filename': filename,'label': label,'size': size}
DF= pd.DataFrame(dict)
DF["id"] = "intensity_"+ (DF["id"]).astype(str)

cols= result.columns
for col in cols:
    for i, path in enumerate(filename):
        if path== col:
            name= DF["id"][i]
            result.rename(columns={col: name}, inplace=True)

cols = result.columns
preordered = ["#CONSENSUS", "charge_cf", "rt_cf", "mz_cf", "quality_cf", "width_cf"]
new_cols = preordered + [c for c in result.columns if c not in preordered]
new_df = result.reindex(columns=new_cols)
new_df.to_csv('results/GNPSexport/FeatureQuantificationTable.txt', sep = '\t', index = False)
print(new_df)

     #CONSENSUS  charge_cf       rt_cf        mz_cf  quality_cf  width_cf  \
0     CONSENSUS          1   91.267878   228.170583    0.000786       NaN   
1     CONSENSUS          1   85.655189   155.081505    0.001448       NaN   
2     CONSENSUS          1  526.949526   432.237990    0.002653       NaN   
3     CONSENSUS          1   71.319476   422.249706    0.000391       NaN   
4     CONSENSUS          1  474.445643   228.195758    0.000731       NaN   
...         ...        ...         ...          ...         ...       ...   
7045  CONSENSUS          1  450.787237   426.264127    0.000031       NaN   
7046  CONSENSUS          2  327.022050   553.285939    0.000114       NaN   
7047  CONSENSUS          1  432.010972  1084.502471    0.000029       NaN   
7048  CONSENSUS          2  111.798745   301.168825    0.000030       NaN   
7049  CONSENSUS          1  340.122249   569.329673    0.000158       NaN   

      intensity_0  intensity_1  intensity_2  intensity_3  ...  intensity_26

6. GNPSExport: This algoriths collects all MS2 spectral information to a single MGF file

In [ ]:
GNPSMGF

7. Create a metadata table

In [4]:
import numpy as np

In [4]:
!(cd data/ && ls *.mzML > filelist.txt)

ls: *.mzML: No such file or directory


In [7]:
header_list = ["filename"]
df=pd.read_csv("data/raw/filelist.txt", names=header_list)
df["ATTRIBUTE_MAPnumber"] = np.arange(len(df))
df["ATTRIBUTE_MAPnumber"]= "MAP" + df["ATTRIBUTE_MAPnumber"].astype(str)
df['ATTRIBUTE_genomeID']=df['filename'].str.extract(r'(NBC_?\d*)')
df['ATTRIBUTE_genomeIDMDNA']=df['filename'].str.extract(r'(MDNAWGS?\d*|MDNA_WGS_?\d*)')
df['ATTRIBUTE_genomeID']=df['ATTRIBUTE_genomeID'].fillna(df['ATTRIBUTE_genomeIDMDNA'])
df["ATTRIBUTE_media"]= df['filename'].str.extract(r'(ISP2|DNPM|FPY12\d*)')
df["ATTRIBUTE_comment"]= df['ATTRIBUTE_genomeID']+"_" + df["ATTRIBUTE_media"]
df=df.drop(columns="ATTRIBUTE_genomeIDMDNA")
df.to_csv("results/GNPSexport/df.tsv", sep='\t')
df

,filename,ATTRIBUTE_MAPnumber,ATTRIBUTE_genomeID,ATTRIBUTE_media,ATTRIBUTE_comment
0,20210826_UMETAB219_POS_DNPM_Plate-1_MDNAWGS11_...,MAP0,MDNAWGS11,DNPM,MDNAWGS11_DNPM
1,20210826_UMETAB219_POS_DNPM_Plate-1_MDNAWGS11_...,MAP1,MDNAWGS11,DNPM,MDNAWGS11_DNPM
2,20210826_UMETAB219_POS_DNPM_Plate-1_MDNAWGS11_...,MAP2,MDNAWGS11,DNPM,MDNAWGS11_DNPM
3,20210826_UMETAB219_POS_DNPM_Plate-1_MDNAWGS11_...,MAP3,MDNAWGS11,DNPM,MDNAWGS11_DNPM
4,20210826_UMETAB219_POS_FPY12_Plate-1_MDNAWGS11...,MAP4,MDNAWGS11,FPY12,MDNAWGS11_FPY12
5,20210826_UMETAB219_POS_FPY12_Plate-1_MDNAWGS11...,MAP5,MDNAWGS11,FPY12,MDNAWGS11_FPY12
6,20210826_UMETAB219_POS_FPY12_Plate-1_MDNAWGS11...,MAP6,MDNAWGS11,FPY12,MDNAWGS11_FPY12
7,20210826_UMETAB219_POS_FPY12_Plate-1_MDNAWGS11...,MAP7,MDNAWGS11,FPY12,MDNAWGS11_FPY12
8,20210826_UMETAB219_POS_ISP2_Plate-1_MDNAWGS11_...,MAP8,MDNAWGS11,ISP2,MDNAWGS11_ISP2
9,20210826_UMETAB219_POS_ISP2_Plate-1_MDNAWGS11_...,MAP9,MDNAWGS11,ISP2,MDNAWGS11_ISP2
